## socket 最底层的API

socket 是一个编程接口，并不一定需要TCP/IP网络

In [1]:
import socket

* socket.AF_INET   ipv4
* socket.AF_INET6  ipv6
* socket.AF_UNIX   unix domain socket

* socket.SOCK_DGRAM    UDP
* socket.SOCK_STREAM    TCP
* socket.SOCK_RAW        既不是UDP也不是TCP。它需要用户空间自己封包

## TCP server/client

In [3]:
so = socket.socket()  # 创建socket实例

In [5]:
so.bind(('127.0.0.1', 1234)) # 绑定地址

In [6]:
so.listen()  # 监听

In [8]:
s, info = so.accept()  # accept 返回一个socket实例和客户端地址

In [10]:
s.recv(1024) # accept返回的socket实例， 可以接受数据

b'1234\x08\x08\x08\r\n'

In [11]:
info # 客户端地址

('127.0.0.1', 36779)

In [14]:
s.recv(1024)

b'abcd\r\n'

1. 初始化socket实例
* bind
* listen
* accept

In [15]:
s.send(b'1234') # accept 返回的socket实例， 可以发送数据

4

In [16]:
s.close()

In [17]:
s, _ = so.accept() # accept 是阻塞的， 直到客户端建立连接

In [18]:
s2, _ = so.accept()

In [20]:
s.send(b'1234\n')

5

In [21]:
s2.send(b'zbcd\n')

5

In [22]:
s2.close()

In [23]:
s.close()

In [24]:
so.close()

In [25]:
import threading

In [74]:
import socket
import threading


class ChatServer:
    def __init__(self, ip='127.0.0.1', port=1234):
        self.sock = socket.socket()
        self.clients = {}
        self.addr = (ip, port)
        self.event = threading.Event()
    
    def recv(self, so, ip, port):
        while not self.event.is_set():
            data = so.recv(1024).decode()
            if data.strip() == '/quit':
                so.close()
                self.clients.pop((ip, port))
                return
            for s in self.clients.values():
                s.send('{}:{}\n{}'.format(ip, port, data).encode())
    
    def accept(self):
        while not self.event.is_set():
            so, (ip, port) = self.sock.accept()
            self.clients[(ip, port)] = so
            threading.Thread(target=self.recv, name='client-{}:{}'.format(ip, port), args=(so, ip, port)).start()

    def start(self):
        self.sock.bind(self.addr)
        self.sock.listen()
        t = threading.Thread(target=self.accept, name='listen', daemon=True)
        try:
            t.start()
            t.join()
        except KeyboardInterrupt:
            self.stop()
    
    def stop(self):
        for so in self.clients.values():
            so.close()
        self.sock.close()
        self.event.set()
        
        

In [80]:
chat_server = ChatServer(port=2345)

In [81]:
chat_server.start()

In [82]:
chat_server.stop()

In [86]:
chat_server.sock.close()

In [87]:
so = socket.socket()

In [89]:
so.bind(('127.0.0.1', 10086))

In [91]:
so.listen()

In [92]:
s, _ = so.accept()

In [102]:
f = s.makefile()

In [108]:
f.readline()

'1234\n'

In [109]:
w = s.makefile('w')

In [111]:
w.write('1234567\n')

8

In [112]:
w.flush()

In [113]:
b = s.makefile('br')

In [114]:
b.readline()

b'abcd\r\n'

In [115]:
s.close()

In [116]:
b.closed

False

In [117]:
b.readable()

True

In [118]:
f.close()

In [119]:
w.close()

In [120]:
b.close()

**makefile生成的类文件对象全部close之后， socket才能真正close**

## TCP client

In [1]:
import socket

In [2]:
so = socket.socket()

In [4]:
conn = so.connect(('127.0.0.1', 1234))

In [7]:
so.send(b'hello\r\n')

7

In [8]:
so.recv(1024)

b'127.0.0.1:36820\nhello\r\n127.0.0.1:36820\nhello\r\n'

In [9]:
so.recv(1024)

b'127.0.0.1:36821\nhello\r\n'

1. 初始化socket.socket实例
* connect

In [10]:
so.send(b'/quit\r\n')

7

In [11]:
so.close()

## UDP Server

In [12]:
so = socket.socket(type=socket.SOCK_DGRAM)

In [13]:
so.bind(('127.0.0.1', 1234))

In [16]:
so.recv(1024)

b'hello\n'

* udp server 不需要也不能执行listen
* udp server 不需要也不能执行accept

只需要两步

1. 初始化（注意type参数）
* bind

In [20]:
so.recv(1024)

b'hello\n'

* 如何直到对端地址
* 如何发送数据

In [22]:
so.recvfrom(1024)

(b'hello\n', ('127.0.0.1', 49026))

In [23]:
so.sendto(b'hello\n', ('127.0.0.1', 49026))

6

* 使用recvfrom接收数据
* 只能以sendto发送数据

In [24]:
so.close()

In [25]:
import threading

In [29]:
import datetime

In [37]:
class ChatServer:
    def __init__(self, host='127.0.0.1', port=1234):
        self.sock = socket.socket(type=socket.SOCK_DGRAM)
        self.host = host
        self.port = port
        self.event = threading.Event()
        self.clients = {}
    
    def recv(self):
        while not self.event.is_set():
            now = datetime.datetime.now()
            data, peer = self.sock.recvfrom(1024)
            data = data.decode().strip()
            if data == '#ping#':
                self.clients[peer] = now
                continue
            disconneted = set()
            for addr, timestamp in self.clients.items():
                if (now - timestamp).total_seconds() > 10:
                    disconneted.add(addr)
                else:
                    self.sock.sendto('{}:{}\n{}'.format(peer[0], peer[1], data).encode(), addr)
            for addr in disconneted:
                self.clients.pop(addr)
        
    
    def start(self):
        self.sock.bind((self.host, self.port))
        self.recv()
        

In [38]:
chat_server = ChatServer(port=1235)

In [39]:
chat_server.start()

OSError: [Errno 98] Address already in use

In [40]:
import socketserver

In [53]:
class EchoHandler(socketserver.BaseRequestHandler):
    def setup(self):
        # 初始化工作 通常情况下不需要重写
        print('setup {}:{}'.format(*self.request.getpeername()))
        super().setup()
    
    def finish(self):
        # 清理工作 通常情况下也不需要重写
        print('finish {}:{}'.format(*self.request.getpeername()))
        super().finish()
    
    def handle(self):
        message = self.request.recv(1024).strip()
        self.request.send(message)

In [54]:
server = socketserver.ThreadingTCPServer(('127.0.0.1', 1234), EchoHandler)

In [55]:
try:
    server.serve_forever()
except KeyboardInterrupt:
    server.shutdown()
    server.server_close()

setup 127.0.0.1:36845
setup 127.0.0.1:36845


In [73]:
class ChatHandler(socketserver.BaseRequestHandler):
    clients = {}
    
    def setup(self):
        self.event = threading.Event()
        self.clients[self.request.getpeername()] = self.request
        super().setup()
    
    def finish(self):
        del self.clients[self.request.getpeername()]
        self.event.set()
        super().finish()
    
    def handle(self):
        while not self.event.is_set():
            message = self.request.recv(1024).strip().decode()
            for so in self.clients.values():
                try:
                    so.send('{1}:{2}\n{0}'.format(message, *self.request.getpeername()).encode())
                except:
                    return

In [74]:
server = socketserver.ThreadingTCPServer(('127.0.0.1', 4000), ChatHandler)

In [75]:
try:
    server.serve_forever()
except KeyboardInterrupt:
    server.server_close()
    server.shutdown()

针对每个连接， server会初始化一个handler， 并且调用handler的handle方法来处理这个连接